In [1]:
---
title: Double Pendulum Simulation
description: An example for numerical simulation of nonlinear ODE's
author: Daning H.
show-code: True
show-prompt: False
---

SyntaxError: invalid syntax (4102755909.py, line 1)

In [2]:
import numpy as np
from scipy.optimize import root_scalar
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import scipy.integrate as integrate

from IPython.display import HTML

In [3]:
G = 9.8  # acceleration due to gravity, in m/s^2
L1 = 1.0  # length of pendulum 1 in m
L2 = 1.0  # length of pendulum 2 in m
M1 = 1.0  # mass of pendulum 1 in kg
M2 = 1.0  # mass of pendulum 2 in kg

def derivs(state, t):
    dydx = np.zeros_like(state)
    dydx[0] = state[1]

    del_ = state[2] - state[0]
    sin_ = np.sin(del_)
    cos_ = np.cos(del_)
    den1 = (M1 + M2)*L1 - M2*L1*cos_*cos_
    dydx[1] = (M2*L1*state[1]*state[1]*sin_*cos_ + M2*G*np.sin(state[2])*cos_ +
               M2*L2*state[3]*state[3]*sin_ - (M1 + M2)*G*np.sin(state[0]))/den1

    dydx[2] = state[3]

    den2 = (L2/L1)*den1
    dydx[3] = (-M2*L2*state[3]*state[3]*sin_*cos_ +
               (M1 + M2)*G*np.sin(state[0])*cos_ -
               (M1 + M2)*L1*state[1]*state[1]*sin_ -
               (M1 + M2)*G*np.sin(state[2]))/den2

    return dydx

# create a time array from 0..100 sampled at 0.05 second steps
dt = 0.05
t = np.arange(0.0, 200, dt)

# th1 and th2 are the initial angles (degrees)
# w10 and w20 are the initial angular velocities (degrees per second)
th1 = 120.0
w1 = 0.0
th2 = -10.0
w2 = 0.0

# initial state
state = np.radians([th1, w1, th2, w2])

# integrate your ODE using scipy.integrate.
y = integrate.odeint(derivs, state, t)

x1 = L1*np.sin(y[:, 0])
y1 = -L1*np.cos(y[:, 0])

x2 = L2*np.sin(y[:, 2]) + x1
y2 = -L2*np.cos(y[:, 2]) + y1

In [15]:
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-2, 2), ylim=(-2, 2))
ax.set_title('Double Pendulum - See when it repeats itself')

line, = ax.plot([], [], 'o-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

def init():
    line.set_data([], [])
    time_text.set_text('')
    return line, time_text

def animate(i):
    thisx = [0, x1[i], x2[i]]
    thisy = [0, y1[i], y2[i]]

    line.set_data(thisx, thisy)
    time_text.set_text(time_template % (i*dt))
    return line, time_text

ani = animation.FuncAnimation(fig, animate, np.arange(1, 1000, 4),#len(y)),
                              interval=100, blit=True, init_func=init)
plt.close(ani._fig)
HTML(ani.to_html5_video())